In [2]:
import pandas as pd
from pythainlp import word_tokenize as tokenize
from pythainlp.util import *
from tqdm.auto import tqdm
import sys
from collections import Counter
sys.path.append('../Cython')
from Method import *
tqdm.pandas()

C:\Users\Alps\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
df = pd.read_csv('../assets/SplitData/2021_10_12.csv')
df

,title,summary,link,published
0,UN Global Compact จัดให้ GC องค์กรต้นแบบด้านคว...,UN Global Compact ชู GC องค์กรต้นแบบด้านความยั...,https://thaipublica.org/2021/10/un-global-comp...,2021_10_12
1,เอ็กโก กรุ๊ป ลงทุนใน “เอเพ็กซ์” ยักษ์ใหญ่พลังง...,เอ็กโก กรุ๊ป ลงทุนใน “เอเพ็กซ์” ยักษ์ใหญ่ด้านพ...,https://thaipublica.org/2021/10/egco-group-inv...,2021_10_12
2,IRPC ร่วมกับ บ.ยูนิคอุตสาหกรรม ส่งมอบถุงพลาสติ...,IRPC ร่วมกับ บ.ยูนิคอุตสาหกรรม ส่งมอบถุงพลาสติ...,https://thaipublica.org/2021/10/irpc-pr-12-10-...,2021_10_12
3,จับสัญญาณลงทุนทำกำไรในโค้งสุดท้ายปี64รับตลาดกา...,เศรษฐกิจทั่วโลกเริ่มส่งสัญญาณการกลับมาฟื้นตัวห...,https://thaipublica.org/2021/10/finvest-direct...,2021_10_12
4,ผลสำรวจสถาบันป๋วยเผยมิติเปราะบางสุดของไทย : คว...,ประเทศไทยกำลังอยู่ในช่วงหัวเลี้ยวหัวต่อที่ต้อง...,https://thaipublica.org/2021/10/pier-report-re...,2021_10_12
...,...,...,...,...
1007,2 ทีมดังพรีเมียร์ลีก เปิดศึกล่า &quot;ลินการ์ด...,2 ทีมร่วมพรีเมียร์ลีก ต่างมีแผนการที่จะเจรจากั...,https://www.thairath.co.th/sport/eurofootball/...,2021_10_12
1008,เปิดประเทศ 1 พ.ย. อย่าหวังนักท่องเที่ยว จะได้ต...,ประเทศไทยเตรียมจะเปิดประเทศ รับนักท่องเที่ยวต่...,https://www.thairath.co.th/scoop/theissue/2217820,2021_10_12
1009,แจงดราม่า ห้องน้ำสำนักงานอัยการจังหวัดนครสวรรค...,รองโฆษกอัยการเเจงประเด็นดราม่า ห้องน้ำอัยการจั...,https://www.thairath.co.th/news/crime/2217817,2021_10_12
1010,ต่าย สายธาร ร้องไห้ ขอระบายความรู้สึก อย่าเห็น...,ทำเอา ต่าย สายธาร นิยมการณ์ ถึงกับต้องร้องไห้ข...,https://www.thairath.co.th/entertain/news/2217810,2021_10_12


In [5]:
NEWMM = "newmm"
ATTACUT = "attacut"
list_column = df.columns.values.tolist()
column_name = list_column[1]

def word_break(news,engine):
    segment = tokenize(news,keep_whitespace=False,engine=engine)
    return segment

In [6]:
atta_list = []
newmm_list = []

def wordBreak(df):
    l = len(df)
    for i in tqdm(range(l)):
        atta_list.append(word_break(df.iloc[i][column_name],ATTACUT))
        newmm_list.append(word_break(df.iloc[i][column_name],NEWMM))

wordBreak(df)

100%|██████████| 1012/1012 [00:57<00:00, 17.57it/s]


In [7]:
def LCS(atta_list, newmm_list):
    l = len(atta_list)
    round = 0
    lcs_atta = []
    lcs_newmm = []
    for i in tqdm(range(l-1)):
        w1_atta = atta_list[i]
        len_w1_atta = len(w1_atta)
        w1_newmm = newmm_list[i]
        len_w1_newmm = len(w1_newmm)
        for j in range(i+1,l):
            round += 1
            w2_atta = atta_list[j]
            len_w2_atta = len(w2_atta)
            w2_newmm = newmm_list[j]
            len_w2_newmm = len(w2_newmm)
            lcs_atta_list,maxl_atta = [],0
            lcs_newmm_list,maxl_newmm = [],0
            if i != j:
                lcs_atta_list,maxl_atta = lcs(w1_atta,w2_atta,len_w1_atta,len_w2_atta)
                lcs_newmm_list,maxl_newmm = lcs(w1_newmm,w2_newmm,len_w1_newmm,len_w2_newmm)
                if lcs_atta_list != []:
                    lcs_atta.extend(lcs_atta_list)
                if lcs_newmm_list != []:
                    lcs_newmm.extend(lcs_newmm_list)

    lcs_atta_counter = Counter(lcs_atta)
    lcs_newmm_counter = Counter(lcs_newmm)
    return lcs_atta_counter, lcs_newmm_counter, round

In [8]:
def LCS_result(data, round):
    # print("result")
    df_result = pd.DataFrame(data.items(), columns=['LCS', 'round'])
    df_result['total_round'] = [round] * len(df_result)
    df_result.sort_values(['round'], ascending=False, inplace=True, ignore_index=True)
    return df_result

In [9]:
lcs_atta, lcs_newmm, round = LCS(atta_list, newmm_list)

100%|██████████| 1011/1011 [02:29<00:00,  6.74it/s]


In [10]:
result_newmm = LCS_result(lcs_newmm,round)
result_newmm[:20]

,LCS,round,total_round
0,[…]Thepost,47268,511566
1,appearedfirstonมติชนออนไลน์.,46677,511566
2,appearedfirston,36117,511566
3,โควิด-19,11927,511566
4,appearedfirstonสำนักข่าวอินโฟเควสท์.,6971,511566
5,[…]อ่านข่าวต้นฉบับ:,5683,511566
6,firstappearedonสำนักข่าวบางกอกทูเดย์.,2434,511566
7,(12ต.ค.,1812,511566
8,12ตุลาคม2564,1723,511566
9,12ต.ค.64,1121,511566


In [19]:
trend = result_newmm[result_newmm["LCS"].str.contains("วันที่13ตุลาคม")]
trend

,LCS,round,total_round
539,วันที่13ตุลาคม2564,4,511566
775,วันที่13ตุลาคม,3,511566


In [12]:
# news = df[df[column_name].str.contains("คิมซอนโฮ", na=False)]
# news

In [13]:
# print(df[column_name].iloc[415])
# print(df[column_name].iloc[436])

In [14]:
result_atta = LCS_result(lcs_atta,round)
result_atta.head(20)

,LCS,round,total_round
0,[…]Thepost,50733,511566
1,[…],48993,511566
2,appearedfirstonมติชนออนไลน์.,46615,511566
3,appearedfirston,11442,511566
4,โควิด-19,6615,511566
5,appearedfirstonสำนักข่าวอินโฟเควสท์.,5989,511566
6,[…]อ่านข่าวต้นฉบับ:,4910,511566
7,ที่ผ่านมา,3590,511566
8,firstappearedonสำนักข่าวบางกอก ทูเดย์.,2320,511566
9,(12ต.ค.,1722,511566


In [15]:
# anchilee = result_atta[result_atta["LCS"].str.contains("แอนชิลี")]
# anchilee

In [16]:
# word_break("คิมซอนโฮ",engine="newmm")